In [1]:
import pyspark 
from pyspark.sql  import SparkSession 
from pyspark.sql.functions import collect_list, col, lit 
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, TimestampType, DateType, ArrayType
import os 
import json 
from pyspark.sql.functions import to_date, when
from urllib.parse import urlparse

In [2]:
aws_access_key = "BKky9mVTl8KoPVCmZZ78"
aws_secret_key = "ZszY0hLzFcJscU8fwAg1bLYSMuWHwpyDpWwfhtmA"
aws_s3_endpoint = "http://18.140.117.93:9000" 
aws_s3_region = "us-east-1" 
nessi_warehouse = "s3a://standardize"
nessi_uri = "http://18.140.117.93:19120/api/v1"
raw_csv_path = "s3a://raw/CRMUSER_ACCOUNTS.csv"

In [3]:
conf = (
    pyspark.SparkConf()
        .setAppName('duytrinh_raw_to_standard_etl')

        .set('spark.jars.packages', 
             'org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.5.0,'
             'org.projectnessie.nessie-integrations:nessie-spark-extensions-3.5_2.12:0.77.1,'
             'software.amazon.awssdk:bundle:2.24.8,'
             'org.apache.iceberg:iceberg-nessie:1.4.0,'
             'org.apache.hadoop:hadoop-aws:3.3.4,'  
             'org.apache.hadoop:hadoop-common:3.3.4,'  
             'io.minio:minio:8.5.0,'
             'org.apache.spark:spark-hadoop-cloud_2.13:3.2.1,'
             'software.amazon.awssdk:url-connection-client:2.24.8,'
             'com.amazonaws:aws-java-sdk-bundle:1.12.207,'
             '')
        .set('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions,org.projectnessie.spark.extensions.NessieSparkSessionExtensions')
        .set('spark.sql.catalog.nessie', 'org.apache.iceberg.spark.SparkCatalog')
        .set('spark.sql.catalog.nessie.uri', nessi_uri)
        .set('spark.sql.catalog.nessie.ref', 'main')
        .set('spark.sql.catalog.nessie.authentication.type', 'NONE')
        .set('spark.sql.catalog.nessie.catalog-impl', 'org.apache.iceberg.nessie.NessieCatalog')
        .set('spark.sql.catalog.nessie.s3.endpoint', aws_s3_endpoint)
        .set('spark.sql.catalog.nessie.warehouse', nessi_warehouse)
        .set('spark.sql.catalog.nessie.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO')
        .set('spark.hadoop.fs.s3a.access.key', aws_access_key)
        .set('spark.hadoop.fs.s3a.secret.key', aws_secret_key)
        .set('spark.hadoop.fs.s3a.endpoint', aws_s3_endpoint)
        .set('spark.hadoop.fs.s3a.region', aws_s3_region)
        .set('spark.hadoop.fs.s3a.path.style.access', 'true')
        .set('spark.hadoop.fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')
        .set('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider')
        .set('spark.hadoop.fs.s3a.connection.ssl.enabled', 'false')
        .set('spark.driver.memory', '512m')
        .set('spark.executor.memory', '512m')
        .set('spark.executor.cores', '1')
)

In [4]:
spark = SparkSession.builder.config(conf=conf).getOrCreate()

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
org.projectnessie.nessie-integrations#nessie-spark-extensions-3.5_2.12 added as a dependency
software.amazon.awssdk#bundle added as a dependency
org.apache.iceberg#iceberg-nessie added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
org.apache.hadoop#hadoop-common added as a dependency
io.minio#minio added as a dependency
org.apache.spark#spark-hadoop-cloud_2.13 added as a dependency
software.amazon.awssdk#url-connection-client added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2fad9ed3-f3f8-4209-81e5-50c234e0062e;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.5.0 in central
	found org.projectnessie.nessie-integrations#nessie-spark-extensions-3.5_2.12;0.77.1 in central
	found

In [5]:
raw_df = spark.read.csv(raw_csv_path, header=True) 
raw_df.show(50)

24/12/11 09:58:58 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


+---------+-------+------------+---------+-------+--------------+------+---------+------+--------------+-----------------------+
|ACCOUNTID|BANK_ID|CORE_CUST_ID|   ORGKEY|ORGTYPE|          NAME|GENDER|CUST_TYPE|STATUS|PRIMARY_SOL_ID|RELATIONSHIPOPENINGDATE|
+---------+-------+------------+---------+-------+--------------+------+---------+------+--------------+-----------------------+
|        1|      1|   113391407|113391407|Account|TRAN T *** HAI|     M|    10001| ACTVE|          1404|             23-02-2018|
|        2|      1|   113391457|113391457|Account|TRAN T *** YEN|     F|    10001| ACTVE|          1003|              3/12/2018|
|        3|      1|   113391483|113391483|Account|TRAN T *** BAC|     F|    10001| ACTVE|          2201|             15-06-2018|
|        4|      1|   105260800|105260800|Account|NGUYEN *** UNG|     M|    10001| ACTVE|          1401|             16-11-2018|
|        5|      1|   106573010|106573010|Account|TRAN N *** ANH|     M|    10001| ACTVE|        